Task:

    Use the provided portfolio position data and BARRA data to conduct a BARRA decomposition of portfolio.


Input:

    Portfolio data:
   
        position: position data
       
        stock_quote: daily market data
       
    BARRA data:
   
        factorloadings
       
        factor returns
       
        individual stock universe
       
        factor covariance matrix
   

Output:

    Factor exposures of the portfolio
    
    The return and risk of the portfolio
    
    (All 43 factors should be taken into account.)
    
   
Introduction to the BARRA factor model:

    The barra factor model is a classic multi-factor model. The model used here is BARRA CNE5 model. It includes 10 style factors, 32 industry factors, and 1 country factor. The total number of factors is 43.
  
    The CNE5 model uses daily individual stock return data to conduct linear regression to obtain returns of these 43 factors, and at the same time obtain a special return, namely the residual.

<h2>read the data</h2>

In [1]:
import pandas as pd
import numpy as np

df_covMat = pd.read_csv('./preprocessed_data/covMat_201209.txt')
df_covMat = df_covMat.iloc[: , :-1]
df_estimatedUniverse = pd.read_csv('./preprocessed_data/estimatedUniverse_201209.txt')
df_estimatedUniverse = df_estimatedUniverse.iloc[: , :-1]
df_factorLoading = pd.read_csv('./preprocessed_data/factorLoading_201209.txt')
df_factorLoading = df_factorLoading.iloc[: , :-1]
df_factorRet = pd.read_csv('./preprocessed_data/factorRet_201209.txt')
df_factorRet = df_factorRet.iloc[: , :-1]
df_stockSpecRet = pd.read_csv('./preprocessed_data/stockSpecRet_201209.txt')
df_stockSpecRet = df_stockSpecRet.iloc[: , :-1]
df_stockSpecRet = pd.read_csv('./preprocessed_data/stockSpecRet_201209.txt')
df_stockSpecRet = df_stockSpecRet.iloc[: , :-1]
df_stockPos = pd.read_csv('./preprocessed_data/stockPos_201209.txt')
df_stockQuote = pd.read_csv('./preprocessed_data/stockQuote_201209.txt', encoding='GB18030')

In [2]:
df_covMat

,NAME,BETA,MOMENTUM,SIZE,EARNYILD,RESVOL,GROWTH,BTOP,LEVERAGE,LIQUIDTY,...,BEV,FOODPROD,HEALTH,BANKS,DVFININS,REALEST,SOFTWARE,HDWRSEMI,UTILITIE,COUNTRY
0,BETA,24.416378,-3.320892,-6.422402,-0.084859,11.338906,0.817447,0.206065,-0.482895,2.720707,...,-7.569756,-9.559699,-1.949545,-4.157674,7.672446,-0.702327,3.385789,9.167096,2.396295,80.699554
1,MOMENTUM,-3.320892,25.470656,5.755354,-0.338766,-2.040214,1.773554,-3.804246,-1.667603,-3.110057,...,13.617771,3.495348,15.250426,0.764909,-13.028471,-2.827582,-0.797225,0.578579,-1.488122,-18.019476
2,SIZE,-6.422402,5.755354,16.942193,2.260937,-8.021882,0.237857,-0.704497,0.419879,-2.208258,...,10.707965,0.396022,-3.401349,0.455506,2.968550,3.921728,-7.706482,-6.560605,-3.385149,-15.770462
3,EARNYILD,-0.084859,-0.338766,2.260937,7.138139,-0.670540,-0.726785,-1.356239,0.063665,-0.151988,...,5.738820,0.103325,-1.503918,-0.681456,4.658891,4.545563,-5.227221,-5.601630,-3.411470,6.350699
4,RESVOL,11.338906,-2.040214,-8.021882,-0.670540,17.768773,1.119906,0.974375,-1.377607,1.116649,...,-6.478615,-3.942410,11.656848,-3.706700,-3.557502,-5.753286,7.762064,7.482384,1.048835,41.392359
5,GROWTH,0.817447,1.773554,0.237857,-0.726785,1.119906,2.732838,-0.621678,-0.575213,-1.483266,...,3.123247,0.331465,4.041352,-0.936948,-4.489466,-2.534256,4.377197,4.395465,-0.694796,2.511592
6,BTOP,0.206065,-3.804246,-0.704497,-1.356239,0.974375,-0.621678,8.091009,1.353413,0.650023,...,-6.852336,-0.886620,-3.073557,-3.734228,8.857086,1.021401,2.439206,-0.866414,1.591776,3.498692
7,LEVERAGE,-0.482895,-1.667603,0.419879,0.063665,-1.377607,-0.575213,1.353413,2.630904,-0.235212,...,0.868888,-0.258193,-4.360198,-1.719570,2.556718,3.131302,-4.624578,-4.023162,-0.496839,-0.607243
8,LIQUIDTY,2.720707,-3.110057,-2.208258,-0.151988,1.116649,-1.483266,0.650023,-0.235212,11.730433,...,-14.587000,-9.781456,-7.191497,-1.437737,8.616781,-1.826411,0.957938,2.897161,2.750438,25.042027
9,SIZENL,2.944470,-0.542788,-2.138286,-1.382815,2.399862,0.149873,-0.412246,-0.330003,2.047265,...,-8.984184,0.953974,0.467033,-2.643774,2.769509,-2.557810,5.010674,4.030207,-0.185222,15.965687


In [3]:
df_estimatedUniverse.head()

,BARRID,SHARES
0,CHNAAA2,317109630
1,CHNAAB2,1961325774
2,CHNAAC2,710914605
3,CHNAAD2,624687999
4,CHNAAE2,934791624


In [4]:
df_factorLoading.head()

,BARRID,SED-CUS,LOCID,NAME,HBETA,PBETA,SRISK%,TRISK%,COUNTRY,BETA,...,LIQUIDTY,SIZENL,INDNAME,IND,PRICE,CAPITALIZATION,YLD%,ESTU,ISOCURR,INTRA_MONTH_ADDITION
0,CHNAAA2,NaN,CN600612,LAOFENGXIANGCOLTD,0.909,0.945,31.008,38.694,1.0,-0.318,...,-0.501,0.974,LEISLUX,19,48.06,1.524029e+10,2.439,1,CNY,0
1,CHNAAA3,NaN,CN600612,LAOFENGXIANGCOLTD,0.909,0.945,31.008,38.694,1.0,-0.318,...,-0.501,0.974,LEISLUX,19,48.06,1.524029e+10,2.439,0,CNY,0
2,CHNAAB2,NaN,CN000012,CSGHOLDINGCOLTD,1.275,1.232,36.545,48.600,1.0,0.979,...,1.488,1.077,CONMAT,3,6.65,1.304282e+10,1.022,1,CNY,0
3,CHNAAB3,NaN,CN000012,CSGHOLDINGCOLTD,1.277,1.232,36.545,48.600,1.0,0.979,...,1.488,1.077,CONMAT,3,6.65,1.304282e+10,1.022,0,CNY,0
4,CHNAAC2,NaN,CN600610,SHANGHAIZHONGYIDA,NaN,0.952,48.555,54.996,1.0,-0.200,...,-1.293,-0.004,COMSERV,13,4.55,3.234661e+09,0.000,1,CNY,0


In [5]:
df_factorRet.head()

,DATE,BETA,MOMENTUM,SIZE,EARNYILD,RESVOL,GROWTH,BTOP,LEVERAGE,LIQUIDTY,...,BEV,FOODPROD,HEALTH,BANKS,DVFININS,REALEST,SOFTWARE,HDWRSEMI,UTILITIE,COUNTRY
0,19990129,-0.000034,-0.000156,-0.001047,-0.001453,-0.001843,-0.000371,0.000657,-0.000174,-0.000766,...,-0.001303,0.000138,-0.000162,0.011584,0.008592,-0.001527,-0.003267,0.001980,0.005043,-0.003677
1,19990201,-0.000914,-0.002338,-0.002505,-0.003567,0.002090,-0.001111,0.003614,-0.000069,-0.003111,...,-0.003089,0.000881,-0.000267,0.006402,0.012147,0.000870,-0.014067,0.000880,0.000133,-0.011066
2,19990202,0.002613,0.003071,0.002757,0.002063,-0.003580,-0.000420,-0.003214,0.000581,0.001388,...,-0.001792,0.003285,-0.003805,-0.008283,-0.011243,0.000091,0.018595,0.005663,-0.002991,0.004302
3,19990203,-0.000896,0.002654,-0.000960,0.000486,-0.000676,-0.000776,-0.001153,-0.000165,-0.002120,...,0.000159,-0.000315,-0.005245,0.002967,0.001380,0.001051,-0.008441,-0.002032,0.001007,-0.009033
4,19990204,0.000582,0.002723,0.000025,-0.000562,0.001052,0.000073,-0.000352,-0.000457,-0.001789,...,0.002562,-0.002620,-0.004118,-0.003212,-0.000400,0.000977,0.006776,-0.001454,-0.002576,-0.014047


In [6]:
df_stockSpecRet.head()

,BARRID,SED-CUS,LOCID,NAME,SPRET%
0,CHNAAA2,NaN,CN600612,LAOFENGXIANGCOLTD,-0.155
1,CHNAAA3,NaN,CN600612,LAOFENGXIANGCOLTD,-0.155
2,CHNAAB2,NaN,CN000012,CSGHOLDINGCOLTD,-0.321
3,CHNAAB3,NaN,CN000012,CSGHOLDINGCOLTD,-0.321
4,CHNAAC2,NaN,CN600610,SHANGHAIZHONGYIDA,1.641


In [7]:
df_stockPos.head()

,InstrumentID,Direction,OffsetFlag,LongPos,TradingDate,StrategyID,AccountID,ShortPos
0,000004.SZE,0,0,4600,2020-12-09,@hljk1alpha,hljk1alpha,0
1,000005.SZE,0,0,15500,2020-12-09,@hljk1alpha,hljk1alpha,0
2,000009.SZE,0,0,7800,2020-12-09,@hljk1alpha,hljk1alpha,0
3,000012.SZE,0,0,8300,2020-12-09,@hljk1alpha,hljk1alpha,0
4,000014.SZE,0,0,6200,2020-12-09,@hljk1alpha,hljk1alpha,0


In [8]:
df_stockQuote.head()

,SecuCode,TradingDay,PrevClosePrice,OpenPrice,HighPrice,LowPrice,ClosePrice,TurnoverVolume,TurnoverValue,SecuMarket,...,Split,Dividend,ActualPlaRatio,PlaPrice,BaseShares,ActualPlaVol,TotalShares,TotalFloatShares,SecuName,ChiName
0,000001.SZE,20201209,18.71,18.79,19.05,18.68,18.71,67722115,1.277484e+09,SZE,...,1.0,0.0,0.0,0.0,0.0,0.0,19405918198,19405752680,平安银行,平安银行股份有限公司
1,000002.SZE,20201209,29.18,29.40,29.44,28.66,28.66,63703506,1.846464e+09,SZE,...,1.0,0.0,0.0,0.0,0.0,0.0,9724196533,9714314935,万科A,万科企业股份有限公司
2,000004.SZE,20201209,25.32,25.35,25.35,24.15,24.40,4317500,1.061222e+08,SZE,...,1.0,0.0,0.0,0.0,0.0,0.0,165052625,83918684,国华网安,深圳国华网安科技股份有限公司
3,000005.SZE,20201209,2.56,2.56,2.57,2.52,2.52,10354195,2.631089e+07,SZE,...,1.0,0.0,0.0,0.0,0.0,0.0,1058536842,1057946242,世纪星源,深圳世纪星源股份有限公司
4,000006.SZE,20201209,5.64,5.64,5.66,5.51,5.52,6764406,3.784777e+07,SZE,...,1.0,0.0,0.0,0.0,0.0,0.0,1349995046,1348307972,深振业A,深圳市振业(集团)股份有限公司


<h2>factor exposures</h2>

In [9]:
industry_index_to_name = ['ENERGY', 'CHEM', 'CONMAT', 'MTLMIN', 'MATERIAL', 'AERODEF', 'BLDPROD', 'CNSTENG', 'ELECEQP', 'INDCONG', 'MACH', 'TRDDIST', 'COMSERV', 'AIRLINE', 'MARINE', 'RDRLTRAN', 'AUTO', 'HOUSEDUR', 'LEISLUX', 'CONSSERV', 'MEDIA', 'RETAIL', 'PERSPRD', 'BEV', 'FOODPROD', 'HEALTH', 'BANKS', 'DVFININS', 'REALEST', 'SOFTWARE', 'HDWRSEMI', 'UTILITIE']
names_of_country_and_style_factors = ['COUNTRY', 'BETA', 'MOMENTUM', 'SIZE', 'EARNYILD', 'RESVOL', 'GROWTH', 'BTOP', 'LEVERAGE', 'LIQUIDTY', 'SIZENL']
df_portfolio_factor_exposures = df_stockPos.merge(df_stockQuote[['SecuCode', 'TotalShares']], left_on='InstrumentID', right_on='SecuCode')
df_portfolio_factor_exposures = df_portfolio_factor_exposures.merge(df_estimatedUniverse, left_on='TotalShares', right_on='SHARES')
df_portfolio_factor_exposures = df_portfolio_factor_exposures.merge(df_factorLoading, on='BARRID')
df_portfolio_factor_exposures = df_portfolio_factor_exposures[['InstrumentID','LongPos', 'TotalShares', 'BARRID', 'LOCID', 'NAME', 'COUNTRY', 'BETA', 'MOMENTUM', 'SIZE', 'EARNYILD', 'RESVOL', 'GROWTH', 'BTOP', 'LEVERAGE', 'LIQUIDTY', 'SIZENL', 'INDNAME', 'IND']]
total_pos = np.sum(df_portfolio_factor_exposures['LongPos'])
portfolio_exposure_data = {}
for factor in names_of_country_and_style_factors:
    portfolio_exposure_data[factor] = np.dot(df_portfolio_factor_exposures['LongPos'], df_portfolio_factor_exposures[factor]) / total_pos
for ind_name in industry_index_to_name:
    portfolio_exposure_data[ind_name] = 0.0
ind_list = list(df_portfolio_factor_exposures['IND'])
pos_list = list(df_portfolio_factor_exposures['LongPos'])
for i in range(len(ind_list)):
    portfolio_exposure_data[industry_index_to_name[ind_list[i]-1]] += pos_list[i]
for ind_name in industry_index_to_name:
    portfolio_exposure_data[ind_name] /= total_pos
pd.DataFrame(portfolio_exposure_data, index=["portfolio_factor_exposure"]).to_csv("portfolio_factor_exposures.csv")

In [10]:
portfolio_exposure_data

{'COUNTRY': 1.0,
 'BETA': 0.04464315045371683,
 'MOMENTUM': -0.8851197446968784,
 'SIZE': -1.987990390609845,
 'EARNYILD': -0.23272948196181054,
 'RESVOL': -0.2517065357978997,
 'GROWTH': -0.3242009427651885,
 'BTOP': 0.29762585941503766,
 'LEVERAGE': -0.24041706534007165,
 'LIQUIDTY': -0.15684746395572916,
 'SIZENL': 0.11698104479015432,
 'ENERGY': 0.025294685438291987,
 'CHEM': 0.0948539668041975,
 'CONMAT': 0.009482539841451268,
 'MTLMIN': 0.047013741228858874,
 'MATERIAL': 0.030922533271554538,
 'AERODEF': 0.006804773869535078,
 'BLDPROD': 0.01581464428981607,
 'CNSTENG': 0.03158131367450543,
 'ELECEQP': 0.0356566402724175,
 'INDCONG': 0.000645495685298615,
 'MACH': 0.08984021441451001,
 'TRDDIST': 0.012655879920145104,
 'COMSERV': 0.03349331353924285,
 'AIRLINE': 0.007333663882650717,
 'MARINE': 0.002028105798325326,
 'RDRLTRAN': 0.023693439689922196,
 'AUTO': 0.04283925816470834,
 'HOUSEDUR': 0.019497634445715353,
 'LEISLUX': 0.042694542196552686,
 'CONSSERV': 0.01094011074373846

<h2>portfolio return</h2>

In [11]:
df_factor_return = df_factorRet.loc[df_factorRet['DATE']==20201209]
portfolio_return = 0.0
for factor in portfolio_exposure_data:
    portfolio_return += float(portfolio_exposure_data[factor] * df_factor_return[factor])
df_special_return = df_portfolio_factor_exposures[['InstrumentID','LongPos','TotalShares','BARRID','LOCID','NAME']].merge(df_stockSpecRet[['BARRID', 'SPRET%']], on='BARRID')
portfolio_return += (np.dot(df_special_return['LongPos'], df_special_return['SPRET%']) / (total_pos * 100.0))

In [12]:
portfolio_return

-0.01974532968387336

<h2>portfolio risk</h2>

In [13]:
factor_ordered_list = ['BETA', 'MOMENTUM', 'SIZE', 'EARNYILD', 'RESVOL', 'GROWTH', 'BTOP', 'LEVERAGE', 'LIQUIDTY', 'SIZENL', 'ENERGY', 'CHEM', 'CONMAT', 'MTLMIN', 'MATERIAL', 'AERODEF', 'BLDPROD', 'CNSTENG', 'ELECEQP', 'INDCONG', 'MACH', 'TRDDIST', 'COMSERV', 'AIRLINE', 'MARINE', 'RDRLTRAN', 'AUTO', 'HOUSEDUR', 'LEISLUX', 'CONSSERV', 'MEDIA', 'RETAIL', 'PERSPRD', 'BEV', 'FOODPROD', 'HEALTH', 'BANKS', 'DVFININS', 'REALEST', 'SOFTWARE', 'HDWRSEMI', 'UTILITIE', 'COUNTRY']
exposure_vector = []
for factor in factor_ordered_list:
    exposure_vector.append(portfolio_exposure_data[factor])
exposure_vector = np.array(exposure_vector)
factor_covariance_matrix = []
for factor in factor_ordered_list:
    factor_covariance_matrix.append(list(df_covMat[factor]))
factor_covariance_matrix = np.array(factor_covariance_matrix)
factor_covariance_matrix = np.divide(factor_covariance_matrix, 10000)
portfolio_variance_part1 = np.dot(exposure_vector, np.dot(factor_covariance_matrix, exposure_vector))
df_special_risk = df_special_return.merge(df_factorLoading[['BARRID', 'SRISK%']], on='BARRID')
portfolio_variance_part2 = np.dot((df_special_risk['SRISK%']/100)**2, (df_special_risk['LongPos'])**2) / (total_pos**2)
portfolio_variance = portfolio_variance_part1 + portfolio_variance_part2
portfolio_standard_deviation = np.sqrt(portfolio_variance)
portfolio_performance = {"portfolio_return":portfolio_return, "portfolio_variance":portfolio_variance, "portfolio_standard_deviation": portfolio_standard_deviation}
pd.DataFrame(portfolio_performance, index=[0]).to_csv("portfolio_performance.csv")

In [14]:
portfolio_performance

{'portfolio_return': -0.01974532968387336,
 'portfolio_variance': 0.0656011843462028,
 'portfolio_standard_deviation': 0.25612728153440195}